## functions

In [1]:
import pandas as pd
import numpy as np
import os
import gc

import addict
import lightgbm as lgb

def save_dataframe(path, dataframe):
    np.save(path + ".data", dataframe.values)
    np.save(path + ".header", dataframe.columns)


def load_dataframe(path):
    data = np.load(path + ".data.npy")
    header = np.load(path + ".header.npy")
    return pd.DataFrame(data=data, columns=header)


def save_dataframe32(path, dataframe, keep=[]):
    col64 = [col_ for col_ in dataframe.columns if col_ in keep]
    col32 = [col_ for col_ in dataframe.columns if col_ not in keep]
    dataframe64 = dataframe[col64]
    dataframe32 = dataframe[col32]
    np.save(path + ".data64", dataframe64.values)
    np.save(path + ".header64", col64)
    np.save(path + ".data32", dataframe32.values.astype(np.float32))
    np.save(path + ".header32", col32)


def load_dataframe32(path, nrows=None):
    path_data32 = path + ".data32.npy"
    path_header32 = path + ".header32.npy"
    path_data64 = path + ".data64.npy"
    path_header64 = path + ".header64.npy"
    result = pd.DataFrame()
    if os.path.exists(path_data32):
        data32 = np.load(path_data32)
        header32 = np.load(path_header32)
        df32 = pd.DataFrame(data=data32, columns=header32)
        result = pd.concat([result, df32], axis=1)
    if os.path.exists(path_data64):
        data64 = np.load(path_data64)
        header64 = np.load(path_header64)
        df64 = pd.DataFrame(data=data64, columns=header64)
        result = pd.concat([result, df64], axis=1)
    if nrows and nrows > 0:
        return result.head(nrows)
    return result

In [2]:
from sklearn.metrics import roc_auc_score

## check data

In [3]:
# nrows = None  
nrows = None

features = list(np.load('./neptune-features/features_246008_1174_0.npy'))
features_nmf = list(np.load('./nmf-features/features-part2.npy'))

train_app = load_dataframe32("./bindata/application_train")
auc_valid_stack = []
pred_valid_stack = []
pred_test_stack = []

for i in range(5):
    train_idx_fn = "./neptune-features/train_idx_{}.npy".format(i)
    valid_idx_fn = "./neptune-features/valid_idx_{}.npy".format(i)

    train_offset, valid_offset, test_offset = i * 4, i * 4 + 1, i * 4 + 3
    
    train_data_fn = "./neptune-features/data_246009_1174_{}".format(train_offset)
    valid_data_fn = "./neptune-features/data_61502_1174_{}".format(valid_offset)
    test_data_fn = "./neptune-features/data_48744_1174_{}".format(test_offset)

    if i == 0:
        train_data_fn = train_data_fn.replace("246009", "246008")
        valid_data_fn = valid_data_fn.replace("61502", "61503")
        
    train_nmf_fn = "./nmf-features/nmf_246009_1174_{}".format(train_offset)
    valid_nmf_fn = "./nmf-features/nmf_61502_1174_{}".format(valid_offset)
    test_nmf_fn = "./nmf-features/nmf_48744_1174_{}".format(test_offset)

    if i == 0:
        train_nmf_fn = train_nmf_fn.replace("246009", "246008")
        valid_nmf_fn = valid_nmf_fn.replace("61502", "61503")
    
    gc.collect()

    train_idx = np.load(train_idx_fn)
    valid_idx = np.load(valid_idx_fn)
    
    if nrows:
        train_idx, valid_idx = train_idx[:nrows].copy(), valid_idx[:nrows].copy()

    train_data = load_dataframe32(train_data_fn, nrows)
    valid_data = load_dataframe32(valid_data_fn, nrows)
    test_data = load_dataframe32(test_data_fn, nrows)
    
    train_nmf = load_dataframe32(train_nmf_fn, nrows)
    valid_nmf = load_dataframe32(valid_nmf_fn, nrows)
    test_nmf = load_dataframe32(test_nmf_fn, nrows)
    
    gc.collect()
    
    X_train = np.hstack([train_data.values, train_nmf.values])
    X_valid = np.hstack([valid_data.values, valid_nmf.values])
    X_test = np.hstack([test_data.values, test_nmf.values])
    y_train = train_app.loc[train_idx].TARGET
    y_valid = train_app.loc[valid_idx].TARGET
    
    print(train_idx_fn, valid_idx_fn)
    print(train_idx.shape, valid_idx.shape)
    print(train_data_fn, train_data.shape)
    print(valid_data_fn, valid_data.shape)
    print(test_data_fn, test_data.shape)

    print(train_nmf_fn, train_nmf.shape)
    print(valid_nmf_fn, valid_nmf.shape)
    print(test_nmf_fn, test_nmf.shape)
    
    print(X_train.shape, y_train.shape)
    print(X_valid.shape, y_valid.shape)
    print(X_test.shape)

./neptune-features/train_idx_0.npy ./neptune-features/valid_idx_0.npy
(246008,) (61503,)
./neptune-features/data_246008_1174_0 (246008, 1174)
./neptune-features/data_61503_1174_1 (61503, 1174)
./neptune-features/data_48744_1174_3 (48744, 1174)
./nmf-features/nmf_246008_1174_0 (246008, 492)
./nmf-features/nmf_61503_1174_1 (61503, 492)
./nmf-features/nmf_48744_1174_3 (48744, 492)
(246008, 1666) (246008,)
(61503, 1666) (61503,)
(48744, 1666)
./neptune-features/train_idx_1.npy ./neptune-features/valid_idx_1.npy
(246009,) (61502,)
./neptune-features/data_246009_1174_4 (246009, 1174)
./neptune-features/data_61502_1174_5 (61502, 1174)
./neptune-features/data_48744_1174_7 (48744, 1174)
./nmf-features/nmf_246009_1174_4 (246009, 492)
./nmf-features/nmf_61502_1174_5 (61502, 492)
./nmf-features/nmf_48744_1174_7 (48744, 492)
(246009, 1666) (246009,)
(61502, 1666) (61502,)
(48744, 1666)
./neptune-features/train_idx_2.npy ./neptune-features/valid_idx_2.npy
(246009,) (61502,)
./neptune-features/data_2

## lightGBM

In [4]:
lgb_params = addict.Dict()
lgb_params.boosting_type = "gbdt"
lgb_params.objective = "binary"
lgb_params.metric = "auc"
lgb_params.learning_rate = 0.018
lgb_params.max_bin = 300
lgb_params.max_depth = -1
lgb_params.num_leaves = 30
lgb_params.min_child_samples = 70
lgb_params.subsample = 1.0
lgb_params.subsample_freq = 1
lgb_params.colsample_bytree = 0.05
lgb_params.min_gain_to_split = 0.5
lgb_params.reg_lambda = 100
lgb_params.reg_alpha = 0.0
lgb_params.scale_pos_weight = 1
lgb_params.is_unbalance = False

In [5]:
nrows = None  
# nrows = 1000

features = list(np.load('./neptune-features/features_246008_1174_0.npy'))
features_nmf = list(np.load('./nmf-features/features-part2.npy'))

train_app = load_dataframe32("./bindata/application_train")
auc_valid_stack = []
pred_valid_stack = []
pred_test_stack = []

for i in range(5):
    train_idx_fn = "./neptune-features/train_idx_{}.npy".format(i)
    valid_idx_fn = "./neptune-features/valid_idx_{}.npy".format(i)

    train_offset, valid_offset, test_offset = i * 4, i * 4 + 1, i * 4 + 3
    
    train_data_fn = "./neptune-features/data_246009_1174_{}".format(train_offset)
    valid_data_fn = "./neptune-features/data_61502_1174_{}".format(valid_offset)
    test_data_fn = "./neptune-features/data_48744_1174_{}".format(test_offset)

    if i == 0:
        train_data_fn = train_data_fn.replace("246009", "246008")
        valid_data_fn = valid_data_fn.replace("61502", "61503")
        
    train_nmf_fn = "./nmf-features/nmf_246009_1174_{}".format(train_offset)
    valid_nmf_fn = "./nmf-features/nmf_61502_1174_{}".format(valid_offset)
    test_nmf_fn = "./nmf-features/nmf_48744_1174_{}".format(test_offset)

    if i == 0:
        train_nmf_fn = train_nmf_fn.replace("246009", "246008")
        valid_nmf_fn = valid_nmf_fn.replace("61502", "61503")
    
    gc.collect()

    train_idx = np.load(train_idx_fn)
    valid_idx = np.load(valid_idx_fn)
    
    if nrows:
        train_idx, valid_idx = train_idx[:nrows].copy(), valid_idx[:nrows].copy()

    train_data = load_dataframe32(train_data_fn, nrows)
    valid_data = load_dataframe32(valid_data_fn, nrows)
    test_data = load_dataframe32(test_data_fn, nrows)
    
    train_nmf = load_dataframe32(train_nmf_fn, nrows)
    valid_nmf = load_dataframe32(valid_nmf_fn, nrows)
    test_nmf = load_dataframe32(test_nmf_fn, nrows)
    
    gc.collect()
    
    X_train = np.hstack([train_data.values, train_nmf.values])
    X_valid = np.hstack([valid_data.values, valid_nmf.values])
    X_test = np.hstack([test_data.values, test_nmf.values])
    y_train = train_app.loc[train_idx].TARGET
    y_valid = train_app.loc[valid_idx].TARGET
    
    print(train_idx_fn, valid_idx_fn)
    print(train_idx.shape, valid_idx.shape)
    print(train_data_fn, train_data.shape)
    print(valid_data_fn, valid_data.shape)
    print(test_data_fn, test_data.shape)

    print(train_nmf_fn, train_nmf.shape)
    print(valid_nmf_fn, valid_nmf.shape)
    print(test_nmf_fn, test_nmf.shape)
    
    print(X_train.shape, y_train.shape)
    print(X_valid.shape, y_valid.shape)
    print(X_test.shape)
    
    lgb_data_train = lgb.Dataset(data=X_train, label=y_train)
    lgb_data_valid = lgb.Dataset(data=X_valid, label=y_valid)
    
    estimator = lgb.train(
        lgb_params,
        lgb_data_train,
        valid_sets=[lgb_data_train, lgb_data_valid],
        valid_names=["data_train", "data_valid"],
        num_boost_round=5000,
        early_stopping_rounds=100,
        verbose_eval=100
    )
    
    pred_valid = estimator.predict(X_valid)
    pred_valid_stack.append(pred_valid)
    
    auc = roc_auc_score(y_valid, pred_valid)
    auc_valid_stack.append(auc)
    print("fold-{},auc:{}".format(i, auc))

    pred_test = estimator.predict(X_test)
    pred_test_stack.append(pred_test)
    
print("kfold-auc, avg:{:.4}, std:{:.2}".format(np.mean(auc_valid_stack), np.std(auc_valid_stack)))

./neptune-features/train_idx_0.npy ./neptune-features/valid_idx_0.npy
(246008,) (61503,)
./neptune-features/data_246008_1174_0 (246008, 1174)
./neptune-features/data_61503_1174_1 (61503, 1174)
./neptune-features/data_48744_1174_3 (48744, 1174)
./nmf-features/nmf_246008_1174_0 (246008, 492)
./nmf-features/nmf_61503_1174_1 (61503, 492)
./nmf-features/nmf_48744_1174_3 (48744, 492)
(246008, 1666) (246008,)
(61503, 1666) (61503,)
(48744, 1666)
Training until validation scores don't improve for 100 rounds.
[100]	data_train's auc: 0.767293	data_valid's auc: 0.762215
[200]	data_train's auc: 0.781909	data_valid's auc: 0.773896
[300]	data_train's auc: 0.79247	data_valid's auc: 0.781019
[400]	data_train's auc: 0.800354	data_valid's auc: 0.785312
[500]	data_train's auc: 0.807104	data_valid's auc: 0.788073
[600]	data_train's auc: 0.813193	data_valid's auc: 0.790403
[700]	data_train's auc: 0.818629	data_valid's auc: 0.792054
[800]	data_train's auc: 0.823743	data_valid's auc: 0.793184
[900]	data_trai

KeyboardInterrupt: 

## save oof & submission

In [ ]:
def calculate_rank(predictions):
    rank = (1 + pd.Series(predictions).rank().values) / (predictions.shape[0] + 1)
    return rank

pred_target = sum([calculate_rank(p_) for p_ in pred_test_stack]) / 5

pred_sample = pd.read_csv("./result/submission-022-blend08.csv")
pred_sample["TARGET"] = pred_target

pred_sample.to_csv("./result/submission-029-lgb.csv", index=False)

In [ ]:
oof_pred = np.zeros(train_app.shape[0])

for i in range(5):
    valid_idx_fn = "./neptune-features/valid_idx_{}.npy".format(i)
    valid_idx = np.load(valid_idx_fn)
    oof_pred[valid_idx] = pred_valid_stack[i]
    
oof_df = train_app[["SK_ID_CURR"]].copy()
oof_df["SK_ID_CURR"] = oof_df.SK_ID_CURR.astype("int")
oof_df["oof_pred"] = oof_pred
oof_df.to_csv("./oof-result/oof-029-lgb.csv", index=False)

In [ ]:
pred_lgb024.head()

In [ ]:
oof_lgb024 = pd.read_csv("./oof-result/oof-024-first.csv")
oof_keras025 = pd.read_csv("./oof-result/oof-024-keras01.csv")


oof_df["oof_lgb028"] = oof_df["oof_pred"]
oof_df["oof_lgb024"] = oof_lgb024["oof_pred"]
oof_df["oof_keras025"] = oof_keras025["oof_pred"]
oof_df["TARGET"] = train_app["TARGET"]

In [ ]:
def rank_avg_and_show_oof_auc(oof_df, columns=["oof_lgb028", "oof_lgb024", "oof_keras025"], weights=[.33, .33, .34]):
    auc_valid_stack = []
    for i in range(5):
        train_idx_fn = "./neptune-features/train_idx_{}.npy".format(i)
        valid_idx_fn = "./neptune-features/valid_idx_{}.npy".format(i)

        train_offset, valid_offset, test_offset = i * 4, i * 4 + 1, i * 4 + 3

        train_data_fn = "./neptune-features/data_246009_1174_{}".format(train_offset)
        valid_data_fn = "./neptune-features/data_61502_1174_{}".format(valid_offset)
        test_data_fn = "./neptune-features/data_48744_1174_{}".format(test_offset)

        if i == 0:
            train_data_fn = train_data_fn.replace("246009", "246008")
            valid_data_fn = valid_data_fn.replace("61502", "61503")

        gc.collect()

        train_idx = np.load(train_idx_fn)
        valid_idx = np.load(valid_idx_fn)

        fold_data = oof_df.loc[valid_idx]
        fold_target = fold_data.TARGET
        
        fold_pred = np.zeros(fold_data.shape[0])
        for col_, w_ in zip(columns, weights):
            fold_pred += w_ * calculate_rank(oof_df.loc[valid_idx][col_])

        auc = roc_auc_score(fold_target, fold_pred)
        auc_valid_stack.append(auc)
        print("fold-{},auc:{}".format(i, auc))

    print("kfold-auc, avg:{:.8}, std:{:.2}".format(np.mean(auc_valid_stack), np.std(auc_valid_stack)))

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.08, .79, .13])

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.09, .79, .12])

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.08, .8, .12])

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.07, .81, .12])

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.06, .82, .12])

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.1, .78, .12])

In [ ]:
rank_avg_and_show_oof_auc(oof_df)

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.1, .8, .1])

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.2, .6, .2])

In [ ]:
rank_avg_and_show_oof_auc(oof_df, weights=[.1, .7, .2])

In [ ]:
oof_df.head()